In [1]:
#import pymongo

In [1]:
import pyMorfologik
from pyMorfologik import Morfologik
from pyMorfologik.parsing import ListParser

In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import codecs
from copy import deepcopy
from sklearn import feature_extraction
#import mpld3

In [3]:
scraping = pd.read_csv('dane.csv')
#ramka danych z treścią w pierwszej kolumnie i adresami w drugiej

In [4]:
documents = scraping['text']
type(documents)

pandas.core.series.Series

In [5]:
#text after an author's name is not important for further analysis. Regular expression below should remove it. 
#It must be done before removing end of line's marks (/n)  
import re
re.compile('[a-zA-Z]',  re.UNICODE) #compiling regex for non-english characters
#for author's description within a domain's name
articles = [re.sub("\\n+_?[A-Z][a-z]+\s[A-Z][a-z]+\s*\\n?[A-Za-z\s]+\.pl.*(\\n.*)*", " ", str(a)) for a in documents]


In [6]:
def words_num(set):
    words=0
    for a in set:
        words += len(a)
    return words
def set_reduction(set1,set2):
    
    return print('Zbiór zmniejszył się o: {} słów. Wynosi: {:.5f} początkowego '
                 .format((words_num(set1) - words_num(set2)), (words_num(set2)/words_num(set1)))) 
set_reduction(documents, articles)

Zbiór zmniejszył się o: 228563 słów. Wynosi: 0.91073 początkowego 


In [7]:
#for author's description without a domain name 
articles = [re.sub("\\n+_?[A-Z]\w+\s[A-Z]\.?\s[A-Z]\w+\s*\\n\\n.*", " ", str(a)) for a in articles]


In [8]:
set_reduction(documents, articles)

Zbiór zmniejszył się o: 228669 słów. Wynosi: 0.91069 początkowego 


In [9]:
#removing local and global urls 
articles = [re.sub("/[a-zA-Z0-9_\-\./]+|htt[^\s]*", " " , str(a)) for a in articles ]

In [10]:
set_reduction(documents, articles)

Zbiór zmniejszył się o: 255920 słów. Wynosi: 0.90005 początkowego 


In [11]:
#clearing articles from special characters and punctualtions   

articles = [re.sub("\\n", " ", str(a)) for a in articles]
articles = [re.sub("[\*|\.|,|:|;|\?|!|#|\(|\)|-]", " ", str(a)) for a in articles]


In [12]:
set_reduction(documents, articles)

Zbiór zmniejszył się o: 255920 słów. Wynosi: 0.90005 początkowego 


In [13]:
#for articles without author deleting the last part of the article containing dates, comments and autopromotional texts

articles2 = [re.sub("data utworzenia.*|Komentarze.*", "", str(a)) for a in articles]

In [14]:
set_reduction(documents, articles2)

Zbiór zmniejszył się o: 660910 słów. Wynosi: 0.74188 początkowego 


In [15]:
reduction=[]
for a in articles:
    m = re.search("(data utworzenia.*|Komentarze.*)", str(a))
    if m:
        found = m.group(1)
        reduction.append(found)
reduction[:5]    

['Komentarze  Brak komentarzy  Dodaj swój komentarz  Autor   Temat   Treść komentarza    []         Jak obniżyć podatek za 2017 rok       PIT 2017   aktualne ulgi i terminy                                Pomoc on line      [e doradca PIT pl]       Jeżeli   masz problemy z fiskusem     możesz także skorzystać   z pomocy profesjonalnego doradcy podatkowego   \\  zadając pytanie lub przedstawiając swoje wątpliwości  www e doradca PIT pl  Jeżeli trafiłeś tu w celu rozliczenia Twojego PITa za ostatni rok  zapraszamy także na aktualny bezpłatny poradnik w zakresie rozliczeń za rok 2007   Twoje pytanie  ',
 'Komentarze  Brak komentarzy  Dodaj swój komentarz  Autor   Temat   Treść komentarza    []         Jak obniżyć podatek za 2017 rok       PIT 2017   aktualne ulgi i terminy                         ',
 'Komentarze    Autor    aneta _ 12 03 2018 21 32 18 _    Temat    ulga  syn uzyskał w 2017 przychód 5091  po odliczeniu kosztów i składek zus wychodzi dochodu 3058 02zł czy przysługuje jeszcze

## Stemming

In [16]:
#stopwords = nltk.corpus.stopwords.words('english')
stopwords = pd.read_csv("stopwords-pl.txt", header=None)

In [17]:
type(stopwords)
stopwords = stopwords.iloc[:,0]
stopwords = stopwords.tolist()

In [18]:
parser = ListParser()
stemmer = Morfologik()
articles = [a.lower() for a in articles]
articles = [re.sub("\d+", "", str(a)) for a in articles]
articles = [nltk.word_tokenize(a, language='polish') for a in articles]
articles = [[w for w in a if w not in stopwords] for a in articles]


     

In [19]:
from collections import Counter
def counting(set):
    c= Counter(set)
    aCounted={}
    for i in set:
        if i not in aCounted:
            aCounted[i]=c[i]

    sort_count = sorted(aCounted.items(), key=lambda item: item[1], reverse=True)

    return pd.DataFrame(sort_count)


In [20]:
articles_stemmed = [stemmer.stem(a, parser) for a in articles]

In [97]:
#stemmming is a long process. If something goes wrong, it'll be better to have a copy :)
x_l=deepcopy(articles_stemmed)

In [39]:
#import string
#corpo = str(" ".join([" ".join(a) for a in articles]))
#corpo

In [40]:
#x=stemmer.stem(corpo, parser)
#amb=[] #ambiguous
#unamb=[] #unambiguous


In [98]:
unamb_corpora=[]
for a in x_l:
    unamb=[]
    for i in range(len(a)):
        if len(a[i][1])==1:
            unamb+=a[i][1]
        elif len(a[i][1])==0:
            unamb+=a[i][0]
    unamb_corpora+=unamb #full match for all corpora      

## Dividing set for ambigous and nonambigous words

In [99]:
for a in x_l:
    stemmed_arts=[]
    amb=[] #ambiguous
    unamb=[] #unambiguous
    for i in range(len(a)):
    
        if len(a[i][1])==1:
            unamb+=a[i][1]
        elif len(a[i][1])==0:
            unamb+=a[i][0]
        else:
            amb.append(a[i]) 
    #unamb_corpora+=unamb #full match for all corpora      
    amb_dict={} #making dictionary with values in a list

    for i in amb:
        amb_dict[i[0]]=[]
        for j in i[1]:
            amb_dict[i[0]].append(j)
    df = counting(unamb)   
    #df_corpora =counting(unamb_corpora)
    
    document_other=[] #collecting remaining words, without a counterpart in unambiguous
    for i in amb_dict:
        for j in range(len(amb_dict[i])):
            if amb_dict[i][j] in unamb:
                #print(amb_dict[i][j])
                unamb.append(amb_dict[i][j])
            elif amb_dict[i][j] in unamb_corpora:
                #print(amb_dict[i][j])
                unamb.append(amb_dict[i][j])
        else:
            document_other.append(amb_dict[i][0]) #when we haven't proper assignment

    document_final=unamb+document_other
        #pd_document = counting(document_final)
    stemmed_arts.append(document_final)

KeyboardInterrupt: 

In [ ]:
#df_corpora[0]
for i in stemmed_arts:
    print("płatnica" in i)


In [26]:
#stemmed_arts2 = [re.sub("\s\w\s", " ",str(a)) for a in stemmed_arts]

In [45]:
counting(stemmed_arts[0])

,0,1
0,podatkowy,10
1,złoty,10
2,pracownik,9
3,zaliczka,9
4,podatek,9
5,dochód,9
6,pracodawca,6
7,kwota,6
8,miesiąc,6
9,praca,5


In [66]:
#print(b[0]) #tuple
#print(b[0][1]) #dict
#print(b) list, len(b)=16
#unamb2=[]

#for i in amb:
#    for j in i[1]:
        #print(i[0],j)
#        unamb2.append({i[0]:j})

amb   

[('aktualności',
  {'aktualności': ['subst:pl:acc:p3+subst:pl:gen:p3+subst:pl:nom:p3+subst:pl:voc:p3'],
   'aktualność': ['subst:pl:acc:f+subst:pl:gen:f+subst:pl:nom:f+subst:pl:voc:f+subst:sg:dat:f+subst:sg:gen:f+subst:sg:loc:f+subst:sg:voc:f']}),
 ('kosztów', {'koszt': ['subst:pl:gen:m3'], 'koszty': ['subst:pl:gen:p3']}),
 ('koszty',
  {'koszt': ['subst:pl:acc:m3+subst:pl:nom:m3+subst:pl:voc:m3'],
   'koszty': ['subst:pl:acc:p3+subst:pl:nom:p3+subst:pl:voc:p3']}),
 ('oświadczenie',
  {'oświadczenie': ['subst:sg:acc:n2+subst:sg:nom:n2+subst:sg:voc:n2'],
   'oświadczyć': ['ger:sg:nom.acc:n2:perf:aff:refl.nonrefl']}),
 ('oświadczenia',
  {'oświadczenie': ['subst:pl:acc:n2+subst:pl:nom:n2+subst:pl:voc:n2+subst:sg:gen:n2'],
   'oświadczyć': ['ger:sg:gen:n2:perf:aff:refl.nonrefl']}),
 ('kosztów', {'koszt': ['subst:pl:gen:m3'], 'koszty': ['subst:pl:gen:p3']}),
 ('zawarcia',
  {'zawarcie': ['subst:pl:acc:n2+subst:pl:nom:n2+subst:pl:voc:n2+subst:sg:gen:n2'],
   'zawrzeć': ['ger:sg:gen:n2:perf:

In [48]:
#unamb_dict

In [50]:
stemmed_arts[2]

['podatkowy',
 'zmiana',
 'podatkowy',
 'pracowniczy',
 'trakt',
 'pracownik',
 'zapominać',
 'takt',
 'wykorzystać',
 'przysługiwać',
 'wysoki',
 'podatkowy',
 'taki',
 'cel',
 'złożyć',
 'zatrudniać',
 'pracodawca',
 'sprawa',
 'naliczać',
 'podatkowy',
 'przynieść',
 'korzyść',
 'podatkowy',
 'początek',
 'moment',
 'praca',
 'trakt',
 'podatkowy',
 'pracownik',
 'spoczywać',
 'obowiązek',
 'poinformować',
 'pracodawca',
 'obliczać',
 'zaliczka',
 'podatek',
 'podatkowy',
 'uzyskać',
 'pracownik',
 'poinformować',
 'pracodawca',
 'podwyższyć',
 'r',
 'y',
 'c',
 'z',
 'a',
 'ł',
 't',
 'o',
 'w',
 'związek',
 'zamieszkiwać',
 'miejscowość',
 'położyć',
 'zakład',
 'praca',
 'brak',
 'dodatek',
 'rozłąka',
 'pracownik',
 'poinformować',
 'pracodawca',
 'zaliczka',
 'obniżać',
 'ponosić',
 'związek',
 'wydatek',
 'imienny',
 'bilet',
 'okresowy',
 'dojazd',
 'praca',
 'miesięczny',
 'roczny',
 'pracownik',
 'miejscowy',
 'złoty',
 '’',
 'złoty',
 'jeden',
 'złoty',
 'pracownik',
 'doj

In [51]:
docs =[]
for a in stemmed_arts:
    b = " ".join(a)
    docs.append(b)


In [63]:
print("pit" not in docs)


True


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

#tfidf
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=no_features, 
    stop_words=stopwords)
tfidf = tfidf_vectorizer.fit_transform(docs)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

#lda
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=no_features, 
    stop_words=stopwords)
tf = tf_vectorizer.fit_transform(docs)
tf_feature_names = tf_vectorizer.get_feature_names()


/home/kodolamacz/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [53]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, 
          random_state=1, alpha=.1, l1_ratio=.5, 
          init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(
    n_topics=no_topics, max_iter=5, learning_method='online',
    learning_offset=50.,random_state=0).fit(tf)

In [54]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
określony dany przychod składanie małżonkowie zatrudniony łącznie spełnienie bądź samotnie
Topic 1:
obowiązujący umowy zawarcie stosowany zastosowanie przychod łącznie pierwszy dany grecki
Topic 2:
pierwszy małżonka płatnica określony dany łącznie grecki list małżonkowie następujący
Topic 3:
złożone następujący płatnica list małżonka rozliczenie pierwszy określony swój dany
Topic 4:
zawarcie łącznie płatnica dany grecki list małżonka małżonkowie następujący obowiązujący
Topic 5:
łącznie płatnica dany grecki list małżonka małżonkowie następujący obowiązujący określony
Topic 6:
przychod zastosowanie grecki umówić pierwsze stosowany obowiązujący umowy zawarcie skalić
Topic 7:
łącznie płatnica dany grecki list małżonka małżonkowie następujący obowiązujący określony
Topic 8:
umowy zawarcie łącznie przychod dany grecki list małżonka małżonkowie następujący
Topic 9:
przychod łącznie płatnica dany grecki list małżonka małżonkowie następujący obowiązujący
Topic 0:
przychod określony za

In [451]:
doc_topic = lda.transform(tf)
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))
    print(scraping.iloc[n,1])


doc: 0 topic: 0

http://www.pit.pl/pit-2017/
doc: 1 topic: 0

http://www.pit.pl/2005/
doc: 2 topic: 0

http://www.pit.pl/roczne2003/
doc: 3 topic: 0

http://www.pit.pl/rozliczenia-roczne-pit-20132014--deklaracje-pit--ulgi-podatkowe-i-odliczenia-20132014--pitpl-12831/
doc: 4 topic: 0

http://www.pit.pl/transakcje-walutowe--inwestycje--podatki-w-praktyce-13694/
doc: 5 topic: 0

http://www.pit.pl/pity_2014_w_2015_za_rok_2015_pity_roczne_2015_rozliczenie_pit_2014_pit_20014/
doc: 6 topic: 0

http://www.pit.pl/rodzina--odpowiedzialnosc-osobista--podatki-w-praktyce-13691/
doc: 7 topic: 0

http://www.pit.pl/pity-roczne/pity-2015-w-2016-za-rok-2015--pity-roczne-2015-rozliczenie-pit-2015--pity-ulgi-odliczenia-14934/
doc: 8 topic: 0

http://www.pit.pl/sprzedaje-za-granica--podatki-a-handel-w-internecie-11353/
doc: 9 topic: 0

http://www.pit.pl/koszty-przesylki--podatki-w-internecie-10257/
doc: 10 topic: 0

http://www.pit.pl/zbycie-praw-autorskich-w-sieci--prywatnie--podatki-a-handel-w-internecie-

In [97]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=titles);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
#plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters

NameError: name 'dist' is not defined